In [ ]:
# import pandas as pd
# import json

In [ ]:
# !ls -l
# ! cd join_easy_health/ && ls -l


In [ ]:
# with open('join_easy_health/2023-05-19_01.58.58-usa_zip_codes_v7.geo.json') as file:
#     data = json.load(file)


# # df = pd.DataFrame(data)

# # If your data has nested structures, you might need to use json_normalize:
# df = pd.json_normalize(data)

# print(df.tail(100))

In [3]:
# Flattenig data from json 
import json
import pandas as pd

with open('join_easy_health/2023-05-19_01.58.58-usa_zip_codes_v7.geo.json') as file:
    data = json.load(file)

rows = []
for feature in data["features"]:
    row = {
        "zip": feature["properties"]["zip"],
        "type": feature["geometry"]["type"],
        "coordinates": feature["geometry"]["coordinates"]
    }
    rows.append(row)

df = pd.DataFrame(rows)
print(df)


         zip     type                                        coordinates
0      00601  Polygon  [[[-66.817631, 18.230404], [-66.836006, 18.209...
1      00602  Polygon  [[[-67.160618, 18.415609], [-67.24024, 18.3787...
2      00603  Polygon  [[[-67.132036, 18.389712], [-67.056497, 18.461...
3      00606  Polygon  [[[-67.019749, 18.195089], [-67.051393, 18.173...
4      00610  Polygon  [[[-67.183076, 18.312805], [-67.22697, 18.2939...
...      ...      ...                                                ...
33078  99923  Polygon  [[[-130.075209, 56.01068], [-130.060918, 55.94...
33079  99925  Polygon  [[[-133.120237, 55.610448], [-133.128404, 55.6...
33080  99926  Polygon  [[[-131.667648, 55.141157], [-131.650294, 55.0...
33081  99927  Polygon  [[[-133.064094, 56.220213], [-133.195776, 56.3...
33082  99929  Polygon  [[[-132.517969, 56.585914], [-132.375058, 56.5...

[33083 rows x 3 columns]


In [4]:
from sqlalchemy import Column, Integer, Sequence, String, create_engine, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm.session import Session

# eng = create_engine("duckdb:///:memory:")
eng = create_engine("duckdb:///database.duckdb")

df.to_sql('draft', eng, if_exists='replace')




-1

In [7]:
with eng.connect() as connection:
    result = connection.execute(text("SELECT * FROM draft"))  # Using text() for SQL statement

    for i, row in enumerate(result):
        if i >= 10:
            break
        
        print(row)

    connection.close()


(0, '00601', 'Polygon', '[[[-66.817631, 18.230404], [-66.836006, 18.209215], [-66.826819, 18.171546], [-66.797216, 18.133093], [-66.768633, 18.134662], [-66.698199, 18.130739], [-66.659408, 18.156636], [-66.692074, 18.215493], [-66.817631, 18.230404]]]')
(1, '00602', 'Polygon', '[[[-67.160618, 18.415609], [-67.24024, 18.378725], [-67.183076, 18.312805], [-67.129994, 18.320652], [-67.132036, 18.389712], [-67.160618, 18.415609]]]')
(2, '00603', 'Polygon', '[[[-67.132036, 18.389712], [-67.056497, 18.461126], [-67.083038, 18.490947], [-67.112641, 18.51292], [-67.168785, 18.478391], [-67.160618, 18.415609], [-67.132036, 18.389712]]]')
(3, '00606', 'Polygon', '[[[-67.019749, 18.195089], [-67.051393, 18.173901], [-66.977896, 18.14408], [-66.923794, 18.14251], [-66.837027, 18.170762], [-66.843151, 18.171546], [-66.898274, 18.188027], [-67.019749, 18.195089]]]')
(4, '00610', 'Polygon', '[[[-67.183076, 18.312805], [-67.22697, 18.29397], [-67.192263, 18.266503], [-67.161639, 18.272781], [-67.1350

In [6]:
df_2 = pd.read_sql('draft', eng)
print(df_2.head(10))



   index    zip          type   
0      0  00601       Polygon  \
1      1  00602       Polygon   
2      2  00603       Polygon   
3      3  00606       Polygon   
4      4  00610       Polygon   
5      5  00612  MultiPolygon   
6      6  00616       Polygon   
7      7  00617       Polygon   
8      8  00622       Polygon   
9      9  00623       Polygon   

                                         coordinates  
0  [[[-66.817631, 18.230404], [-66.836006, 18.209...  
1  [[[-67.160618, 18.415609], [-67.24024, 18.3787...  
2  [[[-67.132036, 18.389712], [-67.056497, 18.461...  
3  [[[-67.019749, 18.195089], [-67.051393, 18.173...  
4  [[[-67.183076, 18.312805], [-67.22697, 18.2939...  
5  [[[[-66.682887, 18.447], [-66.626743, 18.43679...  
6  [[[-66.682887, 18.447], [-66.703303, 18.42895]...  
7  [[[-66.581828, 18.472897], [-66.580807, 18.463...  
8  [[[-67.186138, 18.030288], [-67.195325, 17.965...  
9  [[[-67.113662, 18.028719], [-67.108558, 18.056...  


/home/dvieira/.venvs/vnv-nu/lib/python3.9/site-packages/duckdb_engine/__init__.py:160: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [ ]:
# with eng.connect() as connection: 
#     result = connection.execute(text("SELECT * FROM draft"))
#     rows = result.fetchall()

# for row in rows:
#     print(row)


In [ ]:
# connection.close()

In [20]:
print(df.dtypes)

zip            object
type           object
coordinates    object
dtype: object


In [18]:
import duckdb

duckdb.default_connection.execute("SET GLOBAL pandas_analyze_sample=100000")

con = duckdb.connect('database.duckdb')

con.register('draft', df)

con.query('SELECT * FROM draft')

# con.fetchall()

InvalidInputException: Invalid Input Error: Failed to cast value: Unimplemented type for cast (DOUBLE -> DOUBLE[])